# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
import os
from dotenv import load_dotenv

# The .env file is in a folder named '05_src'
# env_path = os.path.join('C:\DSI\production\scaling_to_production\\01_materials\labs', '.env')
env_path = "C:\DSI\production\scaling_to_production\\01_materials\labs\.env"

# Load the .env file from the specified path
load_dotenv(dotenv_path=env_path)

# Get the variable
PRICE_DATA = os.getenv('PRICE_DATA')
if os.path.exists(PRICE_DATA):
    print("exists")
else:
    print("does not exist")

# Print the variable
print(f"The path to the .env file is: {env_path}")
print(f"The price data is at the following path: {PRICE_DATA}")



exists
The path to the .env file is: C:\DSI\production\scaling_to_production\01_materials\labs\.env
The price data is at the following path: ../../05_src/data/prices/


In [6]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [8]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
print(parquet_files)


['../../05_src/data/prices\\A\\A_2013.parquet', '../../05_src/data/prices\\A\\A_2014.parquet', '../../05_src/data/prices\\A\\A_2015.parquet', '../../05_src/data/prices\\A\\A_2016.parquet', '../../05_src/data/prices\\A\\A_2017.parquet', '../../05_src/data/prices\\A\\A_2018.parquet', '../../05_src/data/prices\\A\\A_2019.parquet', '../../05_src/data/prices\\A\\A_2020.parquet', '../../05_src/data/prices\\A\\A_2021.parquet', '../../05_src/data/prices\\A\\A_2022.parquet', '../../05_src/data/prices\\A\\A_2023.parquet', '../../05_src/data/prices\\A\\A_2024.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2020.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2021.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2022.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2023.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2024.parquet', '../../05_src/data/prices\\ABT\\ABT_2013.parquet', '../../05_src/data/prices\\ABT\\ABT_2014.parquet', '../../05_src/data/prices\\ABT\\ABT_2015.parquet', '../../05_src/dat

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [17]:
# Write your code below.
import numpy as np

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1))
).assign(
        Adj_Close=lambda x: x['Close'] * 1.01  # Adjust as needed
    ).assign(
        Adj_Close_lag = lambda x: x['Adj_Close'].shift(1)
    )
.assign(
    returns = lambda x: (x['Adj_Close']/x['Adj_Close_lag']) - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

print(dd_feat.compute().head(100))

C:\Users\v-sharmashwe\AppData\Local\Temp\ipykernel_40896\823428957.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


                            Date        Open        High         Low  \
ticker                                                                 
A      2024-01-02 00:00:00-05:00  137.491402  139.879263  137.212819   
A      2024-01-03 00:00:00-05:00  137.302382  137.302382  130.407422   
A      2024-01-04 00:00:00-05:00  129.890046  130.835241  129.531866   
A      2024-01-05 00:00:00-05:00  129.342812  131.292910  127.969783   
A      2024-01-08 00:00:00-05:00  129.482108  132.894776  129.153774   
...                          ...         ...         ...         ...   
A      2014-03-20 00:00:00-04:00   36.962780   37.217740   36.714354   
A      2014-03-21 00:00:00-04:00   37.381174   37.773421   36.583603   
A      2014-03-24 00:00:00-04:00   36.662059   36.760121   35.733741   
A      2014-03-25 00:00:00-04:00   36.282891   36.747051   35.942943   
A      2014-03-26 00:00:00-04:00   36.217507   36.603217   36.060609   

             Close   Volume  Dividends  Stock Splits  year   Cl

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [14]:
# Write your code below.
pd_feat = dd_feat.compute()
pd_feat['rolling_avg_return'] = pd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, converting to pandas wasn’t necessary. The moving average return can be directly calculated in Dask. This might even be preferable for larger datasets because Dask can handle computations more efficiently across multiple cores and larger-than-memory data.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.